# New Sprint Doc flow

The following notebook covers the steps to copy a confluence doc, rename it and then send out some slack messages.

In [23]:
from prefect import flow, task, get_run_logger
from atlassian import Confluence
from datetime import datetime, timezone, timedelta
import os
from dotenv import load_dotenv

load_dotenv()

True

In [25]:
@task
def set_variables():
    # Use UTC date for consistency; adjust if you prefer local time
    today = datetime.now(timezone.utc).date()
    end_date = today + timedelta(days=13)
    new_doc_title = f"{today.isoformat()} - {end_date.isoformat()}"

    return {
        "atlassian_key": os.getenv("ATLASSIAN_KEY", ""),
        "atlassian_domain": os.getenv("ATLASSIAN_DOMAIN", ""),
        "atlassian_email": os.getenv("ATLASSIAN_EMAIL", ""),
        "slack_key": os.getenv("SLACK_KEY", ""),
        "confluence_copy_targett_id": os.getenv("CONFLUENCE_COPY_TARGETT_ID", ""),
        "slack_user_id": os.getenv("SLACK_USER_ID", ""),
        "confluence_space_id": os.getenv("CONFLUENCE_SPACE_ID", ""),
        "new_doc_title": new_doc_title,
        "template_id": os.getenv("CONFLUENCE_COPY_TARGETT_ID", "")
    }

In [ ]:
set_variables()

In [ ]:
@task
def copy_template(variables: dict):
    confluence: Confluence = Confluence(
        url=variables["atlassian_domain"],
        username=variables["atlassian_email"],
        password=variables["atlassian_key"],
        cloud=True,
    )
    source_page = confluence.get_page_by_id(page_id=variables["confluence_copy_targett_id"], expand="body.storage,ancestors")
    
    existing = confluence.get_page_by_title(space=variables["confluence_space_id"], title=variables["new_doc_title"])
    if existing:
        return {
            "doc_id": existing["id"],
            "title": existing["title"],
            "created": False
        }

    body_html = source_page["body"]["storage"]["value"]
    parent_id = None
    ancestors = source_page.get("ancestors") or []
    if ancestors:
        parent_id = ancestors[-1]["id"]

    new_page = confluence.create_page(
        space=variables["confluence_space_id"],
        title=variables["new_doc_title"],
        body=body_html,
        parent_id=parent_id
    )

    return {
        "doc_id": new_page["id"],
        "title": new_page["title"],
        "created": True
    }

In [34]:
copy_template(set_variables())

18:46:41.957 | INFO    | Task run 'set_variables' - Finished in state Completed()

18:46:42.821 | ERROR   | atlassian.confluence - Can't find '2025-09-28 - 2025-10-11' page on https://bitovi.atlassian.net/wiki

18:46:44.581 | INFO    | Task run 'copy_template' - Finished in state Completed()

{'doc_id': '1553072155', 'title': '2025-09-28 - 2025-10-11', 'created': True}

In [7]:
@task
def send_slack_update(doc_id: str):
    print(f"Slack: New document created with id {doc_id}")

In [8]:
@task
def send_congrats():
    print("Slack: Congrats! 🎉")

# Main Flow

In [11]:
@flow
def sprint_success_flow():
    logger = get_run_logger()

    vars = set_variables()
    doc_id = None

    try:
        # Step 1 – copy and rename
        copied = copy_template(vars["template_id"])
        renamed = rename_doc(copied["doc_id"], vars["doc_name"])
        doc_id = renamed["doc_id"]

    except Exception as e:
        logger.warning(f"Error during rename: {e}")
        delete_duplicate(copied["doc_id"])
        corrected = find_correct_doc()
        doc_id = corrected["doc_id"]

    # Step 2 – notify in Slack
    send_slack_update(doc_id)
    send_congrats()

In [ ]:
# Biweekly deployment (every 14 days) anchored to a Thursday noon UTC
anchor = datetime(2025, 9, 25, 12, 0, tzinfo=timezone.utc)

sprint_success_flow.deploy(
    name="sprint-scheduler",
    interval=timedelta(days=14),
    anchor_date=anchor,
    tags=["sprint", "biweekly"],
)

In [12]:
sprint_success_flow()

19:48:18.255 | INFO    | Flow run 'lavender-shrimp' - Beginning flow run 'lavender-shrimp' for flow 'sprint-success-flow'

19:48:18.259 | INFO    | Flow run 'lavender-shrimp' - View at http://host.docker.internal:4200/runs/flow-run/f4681d53-947d-4ace-a941-6730172660e2

19:48:18.307 | INFO    | Task run 'set_variables-d24' - Finished in state Completed()

19:48:18.346 | INFO    | Task run 'copy_template-10a' - Finished in state Completed()

19:48:18.372 | ERROR   | Task run 'rename_doc-e9b' - Task run failed with exception: ValueError('Duplicate document exists')
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/prefect/task_engine.py", line 870, in run_context
    yield self
  File "/usr/local/lib/python3.11/site-packages/prefect/task_engine.py", line 1510, in run_task_sync
    engine.call_task_fn(txn)
  File "/usr/local/lib/python3.11/site-packages/prefect/task_engine.py", line 887, in call_task_fn
    result = call_with_parameters(self.task.fn, parameters)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/prefect/utilities/callables.py", line 210, in call_with_parameters
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_74282/1533375476.py", line 6, in rename_doc
    raise ValueError("Duplicate document exists")
ValueError: Duplicate document exists

...

Slack: New document created with id DOC-456


19:48:18.455 | INFO    | Task run 'send_slack_update-f68' - Finished in state Completed()

Slack: Congrats! 🎉


19:48:18.481 | INFO    | Task run 'send_congrats-fc6' - Finished in state Completed()

19:48:18.494 | INFO    | Flow run 'lavender-shrimp' - Finished in state Completed()

Do the following for deployments

```bash
nbdev_export 02_new_sprint_doc.ipynb
```

```bash
prefect deployment build my_flows.py:sprint_success_flow \
    -n "sprint-scheduler" \
    --cron "0 12 * * 4"
```

```bash
prefect deployment apply sprint_success_flow-deployment.yaml
```